LINK TO DATASETS - <br>
[CATEGORY](https://docs.google.com/spreadsheets/d/1J2Wcmti0xy7dECPB0aUhYNOqKYVUiNwbzy_9X04GKm8/edit?usp=sharing)<br>
[POSITIVE AND NEGATIVE](https://docs.google.com/spreadsheets/d/1SQ6I2hnDQknWj9GHrF35OmdFZ_sgScGupcNaAX10agQ/edit?usp=sharing)

# IMPORTING LIBRARIES

In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [2]:
np.random.seed(500)

# ADDING DATA

In [3]:
Corpus = pd.read_csv(r"../input/feedbackreviews/Category - file.csv")

In [4]:
Corpus1 = pd.read_csv(r"../input/feedbackreviews/Positive and Negative - file - file (1).csv")

# DATA PREPROCESSING

In [5]:
Corpus['Reviews'].dropna(inplace=True)
Corpus['Reviews'] = [entry.lower() for entry in Corpus['Reviews']]
Corpus['Reviews']= [word_tokenize(entry) for entry in Corpus['Reviews']]

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['Reviews']):
    
    Final_words = []
  
    word_Lemmatized = WordNetLemmatizer()
    
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    Corpus.loc[index,'text_final'] = str(Final_words)

In [6]:
Corpus1['Reviews'].dropna(inplace=True)

Corpus1['Reviews'] = [entry.lower() for entry in Corpus1['Reviews']]
Corpus1['Reviews']= [word_tokenize(entry) for entry in Corpus1['Reviews']]
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus1['Reviews']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    Corpus1.loc[index,'text_final'] = str(Final_words)

In [7]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['Categories'],test_size=0.2)

In [8]:
Train_X1, Test_X1, Train_Y1, Test_Y1 = model_selection.train_test_split(Corpus1['text_final'],Corpus1['Categories'],test_size=0.2)

In [9]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [10]:
Encoder = LabelEncoder()
Train_Y1 = Encoder.fit_transform(Train_Y1)
Test_Y1 = Encoder.fit_transform(Test_Y1)

In [11]:
Tfidf_vect = TfidfVectorizer(max_features=800)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [12]:
Tfidf_vect1 = TfidfVectorizer(max_features=800)
Tfidf_vect1.fit(Corpus1['text_final'])
Train_X_Tfidf1 = Tfidf_vect1.transform(Train_X1)
Test_X_Tfidf1 = Tfidf_vect1.transform(Test_X1)

# MODEL TRAINING

In [13]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
predictions_NB = Naive.predict(Test_X_Tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

In [14]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

In [15]:
Naive1 = naive_bayes.MultinomialNB()
Naive1.fit(Train_X_Tfidf1,Train_Y1)
predictions_NB1 = Naive1.predict(Test_X_Tfidf1)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB1, Test_Y1)*100)

In [16]:
SVM1 = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM1.fit(Train_X_Tfidf1,Train_Y1)
predictions_SVM1 = SVM1.predict(Test_X_Tfidf1)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM1, Test_Y1)*100)

In [17]:
def prediction_final(arr):
    text = Tfidf_vect.transform(arr)
    text1 = Tfidf_vect1.transform(arr)
    
    k = SVM.predict(text)
    k1 = SVM1.predict(text1)
    
    category = ['Delivery','Product','Payment','Packaging','Customer Service']
    
    final_category = []
    
    for i in range(len(k)):
        cat = ''
        cat = cat + category[k[i]] + ' '
        
        if k1[i] == 0:
        
            if k[i] in (0,1,2,4):
                cat = cat + 'experience was good'


            if k[i] in (3,-1):
                cat = cat + 'was good'
        
        else:
            if k[i] in (0,2,4):
                cat = cat + 'experience was bad'

            if k[i] in (1,-1):
                cat = cat + 'realted issue'

            if k[i] in (3,-1):
                cat = cat + 'was bad'
        
        final_category.append(cat)
    
    for i in range(len(arr)):
        print(f'{s[i]} => Category - {final_category[i]}')

In [23]:
s= ['product was damaged when delivered','delivery was prompt','error while doing payment','packaging was ugly','customer service was amazing and replied instataneously']

prediction_final(s)

# **SAVING MODEL WEIGHTS**

In [27]:
import pickle

filename = './finalized_model0.sav'
pickle.dump(SVM, open(filename, 'wb'))

filename = './finalized_model1.sav'
pickle.dump(SVM1, open(filename, 'wb'))

In [28]:
filename = './TFidf_vector0.sav'
pickle.dump(Tfidf_vect, open(filename, 'wb'))

filename = './Tfidf_vector1.sav'
pickle.dump(Tfidf_vect1, open(filename, 'wb'))

# LOADING MODEL WEIGHTS

In [30]:
svm0 = pickle.load(open('./finalized_model0.sav', 'rb'))
svm1= pickle.load(open('./finalized_model1.sav', 'rb'))
tf0 = pickle.load(open('./TFidf_vector0.sav', 'rb'))
tf1 = pickle.load(open('./Tfidf_vector1.sav', 'rb'))

# PREDICTION ON UNSEEN DATA

In [31]:
def prediction_final(arr):
    text = tf0.transform(arr)
    text1 = tf1.transform(arr)
    
    k = svm0.predict(text)
    k1 = svm1.predict(text1)
    
    category = ['Delivery','Product','Payment','Packaging','Customer Service']
    
    final_category = []
    
    for i in range(len(k)):
        cat = ''
        cat = cat + category[k[i]] + ' '
        
        if k1[i] == 0:
        
            if k[i] in (0,1,2,4):
                cat = cat + 'experience was good'


            if k[i] in (3,-1):
                cat = cat + 'was good'
        
        else:
            if k[i] in (0,2,4):
                cat = cat + 'experience was bad'

            if k[i] in (1,-1):
                cat = cat + 'realted issue'

            if k[i] in (3,-1):
                cat = cat + 'was bad'
        
        final_category.append(cat)
    
    for i in range(len(arr)):
        print(f'{s[i]} => Category - {final_category[i]}')

In [32]:
s= ['product was damaged when delivered','delivery was prompt','error while doing payment','packaging was ugly','customer service was amazing and replied instataneously']

prediction_final(s)